## Date Handling of Input Files

> The data contains periods in CET/CEST format.
	

* we need to have start and end date in one consitent time zone
* we need to split the period into start and end date of the period 

"01/01/2015 00:00:00 - 01/01/2015 00:15:00"


## Setup

All the manipulations and plots in this notebook can be created with standard libraries such as matplotlib, statsmodels etc. 

In [30]:
# Main data packages. 
import numpy as np
import pandas as pd


## Import Data 

The data for this notebook was downloaded from the [meteoblue website](https://www.meteoblue.com/en/weather/archive/export/basel_switzerland_2661604) and consits of weather data for the city of Basel from 2008 till 2020. 

In [31]:
files = [
    "../../data/load/original/GUI_TOTAL_LOAD_DAYAHEAD_201412312300-201512312300.csv",
    "../../data/load/original/GUI_TOTAL_LOAD_DAYAHEAD_201512312300-201612312300.csv",
    "../../data/load/original/GUI_TOTAL_LOAD_DAYAHEAD_201612312300-201712312300.csv",
    "../../data/load/original/GUI_TOTAL_LOAD_DAYAHEAD_201712312300-201812312300 - 1.csv",
    "../../data/load/original/GUI_TOTAL_LOAD_DAYAHEAD_201712312300-201812312300 - 2.csv",
    "../../data/load/original/GUI_TOTAL_LOAD_DAYAHEAD_201812312300-201912312300.csv",
    "../../data/load/original/GUI_TOTAL_LOAD_DAYAHEAD_201912312300-202012312300.csv",
    "../../data/load/original/GUI_TOTAL_LOAD_DAYAHEAD_202012312300-202112312300.csv",
    "../../data/load/original/GUI_TOTAL_LOAD_DAYAHEAD_202112312300-202212312300.csv",
    "../../data/load/original/GUI_TOTAL_LOAD_DAYAHEAD_202212312300-202312312300.csv",
    "../../data/load/original/GUI_TOTAL_LOAD_DAYAHEAD_202312312300-202412312300.csv",
    "../../data/load/original/GUI_TOTAL_LOAD_DAYAHEAD_202412312300-202512312300.csv",
    # "../../data/load/original/GUI_TOTAL_LOAD_DAYAHEAD_202512312300-202612312300.csv",
]

raw_df = pd.concat((pd.read_csv(f, delimiter=",") for f in files), ignore_index=True)


In [32]:
raw_df.shape  #(541275, 6)

(420768, 4)

In [33]:
raw_df.columns

Index(['MTU (CET/CEST)', 'Area', 'Actual Total Load (MW)',
       'Day-ahead Total Load Forecast (MW)'],
      dtype='object')

In [34]:
# Create working copy of dataframe
df_utc_op = raw_df.copy()
df_utc_op = df_utc_op[["MTU (CET/CEST)","Day-ahead Total Load Forecast (MW)", "Actual Total Load (MW)"]].rename(  #"Intraday (MW)","Current (MW)",
    columns={
        'MTU (CET/CEST)': 'period',  
        "Actual Total Load (MW)": "load_actual",
        "Day-ahead Total Load Forecast (MW)": "load_forecast_da",
    }
)
df_utc_op.head()

,period,load_forecast_da,load_actual
0,01/01/2015 00:00 - 01/01/2015 00:15,50136.87,51873.64
1,01/01/2015 00:15 - 01/01/2015 00:30,49545.46,51382.96
2,01/01/2015 00:30 - 01/01/2015 00:45,49067.14,51444.18
3,01/01/2015 00:45 - 01/01/2015 01:00,48475.53,51268.75
4,01/01/2015 01:00 - 01/01/2015 01:15,47452.32,50869.89


In [35]:
df_utc_op.shape

(420768, 3)

In [36]:
# Split "period" into start/end strings
df_utc_op[["period_start", "period_end"]] = df_utc_op["period"].str.split(" - ", n=1, expand=True)
df_utc_op.head()

,period,load_forecast_da,load_actual,period_start,period_end
0,01/01/2015 00:00 - 01/01/2015 00:15,50136.87,51873.64,01/01/2015 00:00,01/01/2015 00:15
1,01/01/2015 00:15 - 01/01/2015 00:30,49545.46,51382.96,01/01/2015 00:15,01/01/2015 00:30
2,01/01/2015 00:30 - 01/01/2015 00:45,49067.14,51444.18,01/01/2015 00:30,01/01/2015 00:45
3,01/01/2015 00:45 - 01/01/2015 01:00,48475.53,51268.75,01/01/2015 00:45,01/01/2015 01:00
4,01/01/2015 01:00 - 01/01/2015 01:15,47452.32,50869.89,01/01/2015 01:00,01/01/2015 01:15


In [37]:

def add_timezone_and_utc(df, col): #="period_start"):
    # State: start in CET
    tz_state = "CET"
    tz_list = []

    # Walk rows in order and flip state if original string contains CET/CEST
    for val in df[col].astype(str):
        if "CEST" in val:
            tz_state = "CEST"
        elif "CET" in val:
            tz_state = "CET"
        tz_list.append(tz_state)

    # Parse datetime from the column (remove any existing timezone text)
    base_dt = pd.to_datetime(
        df[col].astype(str).str.replace(r"\s*\(.*\)$", "", regex=True),
        dayfirst=True,
        errors="coerce"
    )

    # Add labeled version
    # df["period_start_labeled"] = (
    df[col + "_labeled"] = (
        base_dt.dt.strftime("%d/%m/%Y %H:%M:%S") + " (" + pd.Series(tz_list, index=df.index) + ")"
    )

    # Compute UTC by subtracting 1 or 2 hours depending on CET/CEST
    offset_hours = pd.Series(tz_list, index=df.index).map({"CET": 1, "CEST": 2})
    df[col + "_utc"] = base_dt - pd.to_timedelta(offset_hours, unit="h")
    # df["tsp_start_utc"] = base_dt - pd.to_timedelta(offset_hours, unit="h")

    return df


In [38]:
df_utc_op = add_timezone_and_utc(df_utc_op, col="period_start")
df_utc_op = add_timezone_and_utc(df_utc_op, col="period_end")

In [39]:
df_utc_op

,period,load_forecast_da,load_actual,period_start,period_end,period_start_labeled,period_start_utc,period_end_labeled,period_end_utc
0,01/01/2015 00:00 - 01/01/2015 00:15,50136.87,51873.64,01/01/2015 00:00,01/01/2015 00:15,01/01/2015 00:00:00 (CET),2014-12-31 23:00:00,01/01/2015 00:15:00 (CET),2014-12-31 23:15:00
1,01/01/2015 00:15 - 01/01/2015 00:30,49545.46,51382.96,01/01/2015 00:15,01/01/2015 00:30,01/01/2015 00:15:00 (CET),2014-12-31 23:15:00,01/01/2015 00:30:00 (CET),2014-12-31 23:30:00
2,01/01/2015 00:30 - 01/01/2015 00:45,49067.14,51444.18,01/01/2015 00:30,01/01/2015 00:45,01/01/2015 00:30:00 (CET),2014-12-31 23:30:00,01/01/2015 00:45:00 (CET),2014-12-31 23:45:00
3,01/01/2015 00:45 - 01/01/2015 01:00,48475.53,51268.75,01/01/2015 00:45,01/01/2015 01:00,01/01/2015 00:45:00 (CET),2014-12-31 23:45:00,01/01/2015 01:00:00 (CET),2015-01-01 00:00:00
4,01/01/2015 01:00 - 01/01/2015 01:15,47452.32,50869.89,01/01/2015 01:00,01/01/2015 01:15,01/01/2015 01:00:00 (CET),2015-01-01 00:00:00,01/01/2015 01:15:00 (CET),2015-01-01 00:15:00
...,...,...,...,...,...,...,...,...,...
420763,31/12/2025 22:45 - 31/12/2025 23:00,52598.11,48381.94,31/12/2025 22:45,31/12/2025 23:00,31/12/2025 22:45:00 (CET),2025-12-31 21:45:00,31/12/2025 23:00:00 (CET),2025-12-31 22:00:00
420764,31/12/2025 23:00 - 31/12/2025 23:15,51610.61,47631.19,31/12/2025 23:00,31/12/2025 23:15,31/12/2025 23:00:00 (CET),2025-12-31 22:00:00,31/12/2025 23:15:00 (CET),2025-12-31 22:15:00
420765,31/12/2025 23:15 - 31/12/2025 23:30,51120.32,47309.17,31/12/2025 23:15,31/12/2025 23:30,31/12/2025 23:15:00 (CET),2025-12-31 22:15:00,31/12/2025 23:30:00 (CET),2025-12-31 22:30:00
420766,31/12/2025 23:30 - 31/12/2025 23:45,50568.99,47012.07,31/12/2025 23:30,31/12/2025 23:45,31/12/2025 23:30:00 (CET),2025-12-31 22:30:00,31/12/2025 23:45:00 (CET),2025-12-31 22:45:00


In [40]:
df_utc_op.columns

Index(['period', 'load_forecast_da', 'load_actual', 'period_start',
       'period_end', 'period_start_labeled', 'period_start_utc',
       'period_end_labeled', 'period_end_utc'],
      dtype='object')

In [41]:
# Create working copy of dataframe
df_utc_op_temp = df_utc_op.copy()
# select only those values
# df_price = df_price[df_price["Sequence"] != "Sequence Sequence 2"]
 # drop columns
df_utc_op.drop(columns=['period_start', 'period_end', 'period_start_labeled', 'period_end_labeled'], inplace=True)

df_utc_op = df_utc_op.assign(
    date1=lambda x: x["period_start_utc"].dt.date,
    date=lambda x: pd.to_datetime(
        x["date1"],
        format="mixed",
        dayfirst=True,
        errors="coerce"
    ),
    year=lambda x: x["period_start_utc"].dt.year,
    month=lambda x: x["period_start_utc"].dt.month,
    #monthname=lambda x: x["period_start_utc"].dt.month_name(),
    day=lambda x: x["period_start_utc"].dt.day,
    dayofyear=lambda x: x["period_start_utc"].dt.dayofyear,
    hour=lambda x: x["period_start_utc"].dt.hour,
    #week=lambda x: x["period_start_utc"].dt.isocalendar().week.astype(int),
    week=lambda x: x["period_start_utc"].dt.isocalendar().week.astype("Int64"),
    dayofweek=lambda x: x["period_start_utc"].dt.dayofweek,   # 0=Mon ... 6=Sun
    #dayname=lambda x: x["period_start_utc"].dt.day_name(),   

)
df_utc_op.drop(columns=['date1'], inplace=True)

df_utc_op.head()

,period,load_forecast_da,load_actual,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
0,01/01/2015 00:00 - 01/01/2015 00:15,50136.87,51873.64,2014-12-31 23:00:00,2014-12-31 23:15:00,2014-12-31,2014,12,31,365,23,1,2
1,01/01/2015 00:15 - 01/01/2015 00:30,49545.46,51382.96,2014-12-31 23:15:00,2014-12-31 23:30:00,2014-12-31,2014,12,31,365,23,1,2
2,01/01/2015 00:30 - 01/01/2015 00:45,49067.14,51444.18,2014-12-31 23:30:00,2014-12-31 23:45:00,2014-12-31,2014,12,31,365,23,1,2
3,01/01/2015 00:45 - 01/01/2015 01:00,48475.53,51268.75,2014-12-31 23:45:00,2015-01-01 00:00:00,2014-12-31,2014,12,31,365,23,1,2
4,01/01/2015 01:00 - 01/01/2015 01:15,47452.32,50869.89,2015-01-01 00:00:00,2015-01-01 00:15:00,2015-01-01,2015,1,1,1,0,1,3


In [42]:
df_utc_op.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   period            420768 non-null  object        
 1   load_forecast_da  416337 non-null  object        
 2   load_actual       420732 non-null  object        
 3   period_start_utc  420768 non-null  datetime64[ns]
 4   period_end_utc    420768 non-null  datetime64[ns]
 5   date              420768 non-null  datetime64[ns]
 6   year              420768 non-null  int32         
 7   month             420768 non-null  int32         
 8   day               420768 non-null  int32         
 9   dayofyear         420768 non-null  int32         
 10  hour              420768 non-null  int32         
 11  week              420768 non-null  Int64         
 12  dayofweek         420768 non-null  int32         
dtypes: Int64(1), datetime64[ns](3), int32(6), object(3)
memory 

In [43]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["load_forecast_da"].isna()]
rows_with_missing_price.tail(100)

,period,load_forecast_da,load_actual,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
288388,24/03/2022 01:00 - 24/03/2022 01:15,NaN,48686.71,2022-03-24 00:00:00,2022-03-24 00:15:00,2022-03-24,2022,3,24,83,0,12,3
288389,24/03/2022 01:15 - 24/03/2022 01:30,NaN,48676.61,2022-03-24 00:15:00,2022-03-24 00:30:00,2022-03-24,2022,3,24,83,0,12,3
288390,24/03/2022 01:30 - 24/03/2022 01:45,NaN,48202.38,2022-03-24 00:30:00,2022-03-24 00:45:00,2022-03-24,2022,3,24,83,0,12,3
288391,24/03/2022 01:45 - 24/03/2022 02:00,NaN,48092.61,2022-03-24 00:45:00,2022-03-24 01:00:00,2022-03-24,2022,3,24,83,0,12,3
288392,24/03/2022 02:00 - 24/03/2022 02:15,NaN,47856.92,2022-03-24 01:00:00,2022-03-24 01:15:00,2022-03-24,2022,3,24,83,1,12,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
344447,29/10/2023 00:45 - 29/10/2023 01:00,NaN,40105.81,2023-10-28 22:45:00,2023-10-28 23:00:00,2023-10-28,2023,10,28,301,22,43,5
379388,27/10/2024 00:00 - 27/10/2024 00:15,NaN,39742.17,2024-10-26 22:00:00,2024-10-26 22:15:00,2024-10-26,2024,10,26,300,22,43,5
379389,27/10/2024 00:15 - 27/10/2024 00:30,NaN,39289.19,2024-10-26 22:15:00,2024-10-26 22:30:00,2024-10-26,2024,10,26,300,22,43,5
379390,27/10/2024 00:30 - 27/10/2024 00:45,NaN,39130.0,2024-10-26 22:30:00,2024-10-26 22:45:00,2024-10-26,2024,10,26,300,22,43,5


In [44]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["load_actual"].isna()]
rows_with_missing_price

,period,load_forecast_da,load_actual,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
130660,23/09/2018 02:00 - 23/09/2018 02:15,NaN,NaN,2018-09-23 00:00:00,2018-09-23 00:15:00,2018-09-23,2018,9,23,266,0,38,6
130661,23/09/2018 02:15 - 23/09/2018 02:30,NaN,NaN,2018-09-23 00:15:00,2018-09-23 00:30:00,2018-09-23,2018,9,23,266,0,38,6
130662,23/09/2018 02:30 - 23/09/2018 02:45,NaN,NaN,2018-09-23 00:30:00,2018-09-23 00:45:00,2018-09-23,2018,9,23,266,0,38,6
130663,23/09/2018 02:45 - 23/09/2018 03:00,NaN,NaN,2018-09-23 00:45:00,2018-09-23 01:00:00,2018-09-23,2018,9,23,266,0,38,6
130664,23/09/2018 03:00 - 23/09/2018 03:15,NaN,NaN,2018-09-23 01:00:00,2018-09-23 01:15:00,2018-09-23,2018,9,23,266,1,38,6
130665,23/09/2018 03:15 - 23/09/2018 03:30,NaN,NaN,2018-09-23 01:15:00,2018-09-23 01:30:00,2018-09-23,2018,9,23,266,1,38,6
130666,23/09/2018 03:30 - 23/09/2018 03:45,NaN,NaN,2018-09-23 01:30:00,2018-09-23 01:45:00,2018-09-23,2018,9,23,266,1,38,6
130667,23/09/2018 03:45 - 23/09/2018 04:00,NaN,NaN,2018-09-23 01:45:00,2018-09-23 02:00:00,2018-09-23,2018,9,23,266,1,38,6
130668,23/09/2018 04:00 - 23/09/2018 04:15,NaN,NaN,2018-09-23 02:00:00,2018-09-23 02:15:00,2018-09-23,2018,9,23,266,2,38,6
130669,23/09/2018 04:15 - 23/09/2018 04:30,NaN,NaN,2018-09-23 02:15:00,2018-09-23 02:30:00,2018-09-23,2018,9,23,266,2,38,6


In [45]:
# Convert to numeric, turn non-numeric strings into NaN
df_utc_op["load_forecast_da"] = pd.to_numeric(df_utc_op["load_forecast_da"], errors="coerce")
df_utc_op["load_actual"] = pd.to_numeric(df_utc_op["load_actual"], errors="coerce")

rows_with_missing_price = df_utc_op[df_utc_op["load_forecast_da"].isna()]
rows_with_missing_price

,period,load_forecast_da,load_actual,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
129981,16/09/2018 00:15 - 16/09/2018 00:30,NaN,46171.03,2018-09-15 22:15:00,2018-09-15 22:30:00,2018-09-15,2018,9,15,258,22,37,5
129982,16/09/2018 00:30 - 16/09/2018 00:45,NaN,45405.78,2018-09-15 22:30:00,2018-09-15 22:45:00,2018-09-15,2018,9,15,258,22,37,5
129983,16/09/2018 00:45 - 16/09/2018 01:00,NaN,45007.62,2018-09-15 22:45:00,2018-09-15 23:00:00,2018-09-15,2018,9,15,258,22,37,5
129984,16/09/2018 01:00 - 16/09/2018 01:15,NaN,44238.96,2018-09-15 23:00:00,2018-09-15 23:15:00,2018-09-15,2018,9,15,258,23,37,5
129985,16/09/2018 01:15 - 16/09/2018 01:30,NaN,43846.36,2018-09-15 23:15:00,2018-09-15 23:30:00,2018-09-15,2018,9,15,258,23,37,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
344447,29/10/2023 00:45 - 29/10/2023 01:00,NaN,40105.81,2023-10-28 22:45:00,2023-10-28 23:00:00,2023-10-28,2023,10,28,301,22,43,5
379388,27/10/2024 00:00 - 27/10/2024 00:15,NaN,39742.17,2024-10-26 22:00:00,2024-10-26 22:15:00,2024-10-26,2024,10,26,300,22,43,5
379389,27/10/2024 00:15 - 27/10/2024 00:30,NaN,39289.19,2024-10-26 22:15:00,2024-10-26 22:30:00,2024-10-26,2024,10,26,300,22,43,5
379390,27/10/2024 00:30 - 27/10/2024 00:45,NaN,39130.00,2024-10-26 22:30:00,2024-10-26 22:45:00,2024-10-26,2024,10,26,300,22,43,5


In [46]:
df_utc_op.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   period            420768 non-null  object        
 1   load_forecast_da  381297 non-null  float64       
 2   load_actual       385692 non-null  float64       
 3   period_start_utc  420768 non-null  datetime64[ns]
 4   period_end_utc    420768 non-null  datetime64[ns]
 5   date              420768 non-null  datetime64[ns]
 6   year              420768 non-null  int32         
 7   month             420768 non-null  int32         
 8   day               420768 non-null  int32         
 9   dayofyear         420768 non-null  int32         
 10  hour              420768 non-null  int32         
 11  week              420768 non-null  Int64         
 12  dayofweek         420768 non-null  int32         
dtypes: Int64(1), datetime64[ns](3), float64(2), int32(6), objec

In [47]:
group_cols = [
    "date", "year", "month", "day", "dayofyear", "hour", "week", "dayofweek"
]

df_utc_h = (
    df_utc_op
    .groupby(group_cols, as_index=False)
    .agg(
        load_forecast_da=("load_forecast_da", "mean"),
        load_actual=("load_actual", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_hour=("year", "size"),   # count rows per group
    )
)
df_utc_h.head()

,date,year,month,day,dayofyear,hour,week,dayofweek,load_forecast_da,load_actual,period_start_utc,period_end_utc,c_by_hour
0,2014-12-31,2014,12,31,365,23,1,2,49306.2500,51492.3825,2014-12-31 23:00:00,2015-01-01 00:00:00,4
1,2015-01-01,2015,1,1,1,0,1,3,46952.6600,50200.3175,2015-01-01 00:00:00,2015-01-01 01:00:00,4
2,2015-01-01,2015,1,1,1,1,1,3,45751.5975,48431.3625,2015-01-01 01:00:00,2015-01-01 02:00:00,4
3,2015-01-01,2015,1,1,1,2,1,3,45306.1300,46682.6850,2015-01-01 02:00:00,2015-01-01 03:00:00,4
4,2015-01-01,2015,1,1,1,3,1,3,45422.8450,45896.3325,2015-01-01 03:00:00,2015-01-01 04:00:00,4


In [48]:
df_utc_h.tail()

,date,year,month,day,dayofyear,hour,week,dayofweek,load_forecast_da,load_actual,period_start_utc,period_end_utc,c_by_hour
96427,2025-12-31,2025,12,31,365,18,1,2,61429.2525,55093.5625,2025-12-31 18:00:00,2025-12-31 19:00:00,4
96428,2025-12-31,2025,12,31,365,19,1,2,58695.5025,52129.1650,2025-12-31 19:00:00,2025-12-31 20:00:00,4
96429,2025-12-31,2025,12,31,365,20,1,2,55991.9725,50470.0175,2025-12-31 20:00:00,2025-12-31 21:00:00,4
96430,2025-12-31,2025,12,31,365,21,1,2,53693.2025,49173.3875,2025-12-31 21:00:00,2025-12-31 22:00:00,4
96431,2025-12-31,2025,12,31,365,22,1,2,50790.7075,47209.1025,2025-12-31 22:00:00,2025-12-31 23:00:00,4


In [49]:
df_utc_h["c_by_hour"].value_counts()

c_by_hour
4    87672
8     8760
Name: count, dtype: int64

In [50]:
df_utc_h[df_utc_h["c_by_hour"] > 4]

,date,year,month,day,dayofyear,hour,week,dayofweek,load_forecast_da,load_actual,period_start_utc,period_end_utc,c_by_hour
26304,2017-12-31,2017,12,31,365,23,52,6,48354.8750,51975.6725,2017-12-31 23:00:00,2018-01-01 00:00:00,8
26305,2018-01-01,2018,1,1,1,0,1,0,46139.5075,50553.1575,2018-01-01 00:00:00,2018-01-01 01:00:00,8
26306,2018-01-01,2018,1,1,1,1,1,0,44956.5850,49213.2300,2018-01-01 01:00:00,2018-01-01 02:00:00,8
26307,2018-01-01,2018,1,1,1,2,1,0,44586.4950,48267.5650,2018-01-01 02:00:00,2018-01-01 03:00:00,8
26308,2018-01-01,2018,1,1,1,3,1,0,44864.9650,48099.0100,2018-01-01 03:00:00,2018-01-01 04:00:00,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35059,2018-12-31,2018,12,31,365,18,1,0,52685.7600,55245.6875,2018-12-31 18:00:00,2018-12-31 19:00:00,8
35060,2018-12-31,2018,12,31,365,19,1,0,49721.5875,51483.6750,2018-12-31 19:00:00,2018-12-31 20:00:00,8
35061,2018-12-31,2018,12,31,365,20,1,0,47894.3925,49566.3625,2018-12-31 20:00:00,2018-12-31 21:00:00,8
35062,2018-12-31,2018,12,31,365,21,1,0,46091.6500,48976.0925,2018-12-31 21:00:00,2018-12-31 22:00:00,8


In [51]:
group_cols = [
    "date", "year", "month", "day", "dayofyear", "week", "dayofweek"
]

df_utc_d = (
    df_utc_h
    .groupby(group_cols, as_index=False)
    .agg(
        load_forecast_da=("load_forecast_da", "mean"),
        load_actual=("load_actual", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_day=("year", "size"),   # count rows per group
    )
)
df_utc_d.head()

,date,year,month,day,dayofyear,week,dayofweek,load_forecast_da,load_actual,period_start_utc,period_end_utc,c_by_day
0,2014-12-31,2014,12,31,365,1,2,49306.250000,51492.382500,2014-12-31 23:00:00,2015-01-01,1
1,2015-01-01,2015,1,1,1,1,3,52679.205000,52744.743125,2015-01-01 00:00:00,2015-01-02,24
2,2015-01-02,2015,1,2,2,1,4,56814.259583,60347.033958,2015-01-02 00:00:00,2015-01-03,24
3,2015-01-03,2015,1,3,3,1,5,54239.152500,56948.717396,2015-01-03 00:00:00,2015-01-04,24
4,2015-01-04,2015,1,4,4,1,6,50884.030000,55061.048958,2015-01-04 00:00:00,2015-01-05,24


In [52]:
df_utc_d.tail()

,date,year,month,day,dayofyear,week,dayofweek,load_forecast_da,load_actual,period_start_utc,period_end_utc,c_by_day
4014,2025-12-27,2025,12,27,361,52,5,49537.680521,50653.391875,2025-12-27,2025-12-28 00:00:00,24
4015,2025-12-28,2025,12,28,362,52,6,50565.321667,49057.378438,2025-12-28,2025-12-29 00:00:00,24
4016,2025-12-29,2025,12,29,363,1,0,59703.049688,53515.487708,2025-12-29,2025-12-30 00:00:00,24
4017,2025-12-30,2025,12,30,364,1,1,59057.040208,52282.361771,2025-12-30,2025-12-31 00:00:00,24
4018,2025-12-31,2025,12,31,365,1,2,55744.854783,51670.763043,2025-12-31,2025-12-31 23:00:00,23


In [53]:
df_utc_d["c_by_day"].value_counts()

c_by_day
24    4017
1        1
23       1
Name: count, dtype: int64

In [54]:
group_cols = [
    "year"
]

df_utc_y = (
    df_utc_d
    .groupby(group_cols, as_index=False)
    .agg(
        load_forecast_da=("load_forecast_da", "mean"),
        load_actual=("load_actual", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_year=("year", "size"),   # count rows per group
    )
)
df_utc_y.head(20)

,year,load_forecast_da,load_actual,period_start_utc,period_end_utc,c_by_year
0,2014,49306.250000,51492.382500,2014-12-31 23:00:00,2015-01-01 00:00:00,1
1,2015,61643.969985,64542.627632,2015-01-01 00:00:00,2016-01-01 00:00:00,365
2,2016,61625.396147,65006.784839,2016-01-01 00:00:00,2017-01-01 00:00:00,366
3,2017,63600.526306,65599.377342,2017-01-01 00:00:00,2018-01-01 00:00:00,365
4,2018,62761.935515,63931.683227,2018-01-01 00:00:00,2019-01-01 00:00:00,365
5,2019,55177.544094,57365.626309,2019-01-01 00:00:00,2020-01-01 00:00:00,365
6,2020,54641.353912,55817.274674,2020-01-01 00:00:00,2021-01-01 00:00:00,366
7,2021,56298.691198,58189.695490,2021-01-01 00:00:00,2022-01-01 00:00:00,365
8,2022,56288.773441,55626.630707,2022-01-01 00:00:00,2023-01-01 00:00:00,365
9,2023,53043.928516,52863.061880,2023-01-01 00:00:00,2024-01-01 00:00:00,365


In [55]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["load_forecast_da"].isna()]
rows_with_missing_price

,period,load_forecast_da,load_actual,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
129981,16/09/2018 00:15 - 16/09/2018 00:30,NaN,46171.03,2018-09-15 22:15:00,2018-09-15 22:30:00,2018-09-15,2018,9,15,258,22,37,5
129982,16/09/2018 00:30 - 16/09/2018 00:45,NaN,45405.78,2018-09-15 22:30:00,2018-09-15 22:45:00,2018-09-15,2018,9,15,258,22,37,5
129983,16/09/2018 00:45 - 16/09/2018 01:00,NaN,45007.62,2018-09-15 22:45:00,2018-09-15 23:00:00,2018-09-15,2018,9,15,258,22,37,5
129984,16/09/2018 01:00 - 16/09/2018 01:15,NaN,44238.96,2018-09-15 23:00:00,2018-09-15 23:15:00,2018-09-15,2018,9,15,258,23,37,5
129985,16/09/2018 01:15 - 16/09/2018 01:30,NaN,43846.36,2018-09-15 23:15:00,2018-09-15 23:30:00,2018-09-15,2018,9,15,258,23,37,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
344447,29/10/2023 00:45 - 29/10/2023 01:00,NaN,40105.81,2023-10-28 22:45:00,2023-10-28 23:00:00,2023-10-28,2023,10,28,301,22,43,5
379388,27/10/2024 00:00 - 27/10/2024 00:15,NaN,39742.17,2024-10-26 22:00:00,2024-10-26 22:15:00,2024-10-26,2024,10,26,300,22,43,5
379389,27/10/2024 00:15 - 27/10/2024 00:30,NaN,39289.19,2024-10-26 22:15:00,2024-10-26 22:30:00,2024-10-26,2024,10,26,300,22,43,5
379390,27/10/2024 00:30 - 27/10/2024 00:45,NaN,39130.00,2024-10-26 22:30:00,2024-10-26 22:45:00,2024-10-26,2024,10,26,300,22,43,5


In [56]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["load_actual"].isna()]
rows_with_missing_price

,period,load_forecast_da,load_actual,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
130660,23/09/2018 02:00 - 23/09/2018 02:15,NaN,NaN,2018-09-23 00:00:00,2018-09-23 00:15:00,2018-09-23,2018,9,23,266,0,38,6
130661,23/09/2018 02:15 - 23/09/2018 02:30,NaN,NaN,2018-09-23 00:15:00,2018-09-23 00:30:00,2018-09-23,2018,9,23,266,0,38,6
130662,23/09/2018 02:30 - 23/09/2018 02:45,NaN,NaN,2018-09-23 00:30:00,2018-09-23 00:45:00,2018-09-23,2018,9,23,266,0,38,6
130663,23/09/2018 02:45 - 23/09/2018 03:00,NaN,NaN,2018-09-23 00:45:00,2018-09-23 01:00:00,2018-09-23,2018,9,23,266,0,38,6
130664,23/09/2018 03:00 - 23/09/2018 03:15,NaN,NaN,2018-09-23 01:00:00,2018-09-23 01:15:00,2018-09-23,2018,9,23,266,1,38,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...
169067,28/10/2018 02:45 (CET) - 28/10/2018 03:00,NaN,NaN,2018-10-28 01:45:00,2018-10-28 02:00:00,2018-10-28,2018,10,28,301,1,43,6
169296,30/10/2018 12:00 - 30/10/2018 12:15,71879.97,NaN,2018-10-30 11:00:00,2018-10-30 11:15:00,2018-10-30,2018,10,30,303,11,44,1
169297,30/10/2018 12:15 - 30/10/2018 12:30,71884.10,NaN,2018-10-30 11:15:00,2018-10-30 11:30:00,2018-10-30,2018,10,30,303,11,44,1
169298,30/10/2018 12:30 - 30/10/2018 12:45,71734.92,NaN,2018-10-30 11:30:00,2018-10-30 11:45:00,2018-10-30,2018,10,30,303,11,44,1


## saving our files by hour

In [58]:
df_utc_h.to_csv("../../data_cleaned/by_source/02_LOAD.csv", index=False)